In [1]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [2]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.neural_network import MLPClassifier
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re

Using TensorFlow backend.


In [0]:
fields = ['Event 1', 'Event 2', 'Label']

df_train = pd.read_csv('P1_training_set.csv', skipinitialspace=True, usecols=fields)
df_test = pd.read_csv('P1_testing_set.csv', skipinitialspace=True, usecols=fields)

In [0]:
from nltk.stem.snowball import SnowballStemmer
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text
  
# apply the above function to df['text']
df_train['Event 1'] = df_train['Event 1'].map(lambda x: clean_text(x))
df_train['Event 2'] = df_train['Event 2'].map(lambda x: clean_text(x))

df_test['Event 1'] = df_test['Event 1'].map(lambda x: clean_text(x))
df_test['Event 2'] = df_test['Event 2'].map(lambda x: clean_text(x))

In [0]:
temp_train = np.column_stack((df_train['Event 1'], df_train['Event 2']))
temp_test = np.column_stack((df_test['Event 1'], df_test['Event 2']))

sent_train = []
sent_test = []

for i in range(len(temp_train)):
  sent_train.append(temp_train[i][0] + " " + temp_train[i][1])
  
for i in range(len(temp_test)):
  sent_test.append(temp_test[i][0] + " " + temp_test[i][1])
  
sent_train = np.array(sent_train)
sent_test = np.array(sent_test)

In [0]:
data_train = pd.DataFrame({'Event Pairs': sent_train[0:], 'Label': df_train['Label']})
data_test = pd.DataFrame({'Event Pairs': sent_test[0:], 'Label': df_test['Label']})

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [8]:
train_list = data_train['Event Pairs'].tolist()

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  x_train = session.run(embed(train_list))

test_list = data_test['Event Pairs'].tolist()

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  x_test = session.run(embed(test_list))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
labels = df_train['Label']

## create model
model_NN = Sequential()
model_NN.add(Embedding(20000, 512, input_length=512))
model_NN.add(Dropout(0.2))
model_NN.add(Conv1D(64, 5, activation='relu'))
model_NN.add(MaxPooling1D(pool_size=4))
model_NN.add(Flatten())
model_NN.add(Dense(3, activation='softmax'))
model_NN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_NN.fit(x_train, labels, epochs = 100)

model_NN.evaluate(x_test, y = df_test['Label'])

Epoch 1/100
14358/14358 [==============================] - 19s 1ms/step - loss: 0.9917 - acc: 0.4422
Epoch 2/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9849 - acc: 0.4402
Epoch 3/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9850 - acc: 0.4418
Epoch 4/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9861 - acc: 0.4400
Epoch 5/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9845 - acc: 0.4304
Epoch 6/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9843 - acc: 0.4418
Epoch 7/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9833 - acc: 0.4352
Epoch 8/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9833 - acc: 0.4379
Epoch 9/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9879 - acc: 0.4352
Epoch 10/100
14358/14358 [==============================] - 16s 1ms/step - loss: 0.9836 - a

[0.971889308388752, 0.4420334781391312]